In [162]:
import time
import folium
import osmnx as ox
import networkx as nx

weight = "length"

In [163]:
G = ox.graph_from_place("Makati, Poblacion", network_type="drive")
nodes, edges = ox.graph_to_gdfs(G)

# NCR to test the Dijkstra
metro_G = ox.io.load_graphml(filepath="data/metro_drive.graphml")
metro_nodes, metro_edges = ox.graph_to_gdfs(metro_G)

In [173]:
test_orig = ox.nearest_nodes(metro_G, 120.99169, 14.51015)
test_dest = ox.nearest_nodes(metro_G, 121.0643, 14.6548)

In [3]:
print(f"Number of nodes: {len(G.nodes)} and Number of edges: {len(G.edges)}")

Number of nodes: 267 and Number of edges: 612


In [156]:
# Sample nodes
source = 32546634
target = 3650224015 # 248035797 

## Dijkstra test implementation

In [150]:
# Taken from https://gist.github.com/qpwo/cda55deee291de31b50d408c1a7c8515 (just reworked the search on neighbors)
from queue import PriorityQueue

def dijkstra(G, start, goal):
    """ Uniform-cost search / dijkstra """
    visited = set()
    cost = {start: 0}
    parent = {start: None}
    todo = PriorityQueue()
  
    todo.put((0, start))
    while todo:
        while not todo.empty():
            _, vertex = todo.get() # finds lowest cost vertex
            # loop until we get a fresh vertex
            if vertex not in visited: break
        else: # if todo ran out
            break # quit main loop
        visited.add(vertex)
        if vertex == goal:
            break
        
        # Explored the MultiDiGraph documentation from NetworkX, and managed to get the neighbor and edge lenghts using out_edges
        for _, neighbor, length in G.out_edges(vertex, data="length"):
            if neighbor in visited: continue
            old_cost = cost.get(neighbor, float('inf')) # default to infinity
            new_cost = cost[vertex] + length
            if new_cost < old_cost:
                todo.put((new_cost, neighbor))
                cost[neighbor] = new_cost
                parent[neighbor] = vertex

    return parent

def make_path(parent, goal):
    if goal not in parent:
        return None
    v = goal
    path = []
    while v is not None: # root has null parent
        path.append(v)
        v = parent[v]
    return path[::-1]

## Compare performance of Dijkstra implementation from osmNX/NetworkX

Noticed that the `nx.dijkstra_path` from NetworkX gives out a different path compared to osmNX (which uses a different Dijkstra function in the NetworkX library). But the `ox.shortest_path` route matches our implementation's route: all is good

In [192]:
# Run the Dijkstra
start = time.perf_counter() 
parent = dijkstra(metro_G, test_orig, test_dest)
test_route = make_path(parent, test_dest)
end = time.perf_counter() 

print(f"The time it took to run Dijkstra {end - start}")

The time it took to run Dijkstra 1.093954499985557


In [191]:
# Run the Dijkstra from NetworkX
start = time.perf_counter() 
parent = ox.shortest_path(metro_G, test_orig, test_dest)
end = time.perf_counter() 

print(f"The time it took to run Dijkstra {end - start}")

The time it took to run Dijkstra 0.9670166000141762


## Plot the route

In [197]:
# Plot the nodes and edges, and the shortest path route from our sample points
route = ox.shortest_path(metro_G, test_orig, test_dest) # To compare with osmnx shortest path
route_edges = ox.routing.route_to_gdf(metro_G, route, weight=weight)

test_route_edges = ox.routing.route_to_gdf(metro_G, test_route, weight=weight) # test_route holds our Dijktstra implementation's

m = test_route_edges.explore(color="skyblue", tiles="cartodbdarkmatter", style_kwds={"weight": 5, "opacity": 0.5})
route_edges.explore(m=m, color ="pink")